In [1]:
!pip install jsonlines
import jsonlines

In [4]:
def read_jsonl(file_path):
    data = []
    with jsonlines.open(file_path, mode='r') as reader:
        for item in reader:
            data.append(item)
    return data

In [5]:
flatten_laws = read_jsonl('/kaggle/input/law-qa/flatten_law_corpus.json')

In [6]:
print(len(flatten_laws))

28314


In [7]:
import re
def chunk_text_with_subheadings(text):
    if re.search(r'\d+\.', text) or re.search(r'[a-z]\)', text, flags=re.IGNORECASE):
        chunks = re.split(r'\n(?=\d+\.)', text)
        final_chunks = [chunk.strip() for chunk in chunks]
    else:
        final_chunks = [text.strip()]
    
    return final_chunks

In [8]:
for corpus in flatten_laws:
    corpus['text'] = '\n'.join([str(content_part) for content_part in corpus['content']])
    del corpus['content']  

In [9]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('bkai-foundation-models/vietnamese-bi-encoder')

def count_tokens(text):
    tokens = tokenizer.tokenize(text)
    return len(tokens)

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

In [10]:
chunk_id_mapping = {}
chunk_no_segmented_corpus = []
max_chunk_size = 255
num_over_input = 0
longest_chunk = 0

for doc in flatten_laws:
    list_chunk = chunk_text_with_subheadings(doc['text'])
    chunk_postfix = 1
    list_chunk_id = []
    
    for chunk in list_chunk:
        if len(chunk) > 5:
            text = ""

            text = doc['name'] + " : " + chunk
            chunk_id = doc['article_id'] + "_" + str(chunk_postfix)
            
            list_chunk_id.append(chunk_id)
            
            chunk_no_segmented_corpus.append({
            'doc_id' : doc['article_id'],
            'chunk_id' : chunk_id,
            'title' : doc['name'],
            'chunk' : chunk,
            'text' : text
            })
            
            chunk_postfix += 1
            
            if count_tokens(text) > longest_chunk:
                longest_chunk = count_tokens(text)
            
            if count_tokens(text) > max_chunk_size:
                num_over_input += 1
                
    chunk_id_mapping[doc['article_id']] = list_chunk_id

In [11]:
chunk_no_segmented_corpus[100]

{'doc_id': '32/2024/qh15_21',
 'chunk_id': '32/2024/qh15_21_3',
 'title': 'Điều 21. Ban kiểm soát của ngân hàng chính sách',
 'chunk': '3. Số lượng, việc bổ nhiệm, miễn nhiệm thành viên Ban kiểm soát; cơ cấu, nhiệm vụ, quyền hạn của Ban kiểm soát do Chính phủ quy định.',
 'text': 'Điều 21. Ban kiểm soát của ngân hàng chính sách : 3. Số lượng, việc bổ nhiệm, miễn nhiệm thành viên Ban kiểm soát; cơ cấu, nhiệm vụ, quyền hạn của Ban kiểm soát do Chính phủ quy định.'}

In [12]:
print(num_over_input)
print(longest_chunk)
print(len(chunk_no_segmented_corpus))

3803
8140
92351


In [2]:
import json
no_segment_queries = []

with open("/kaggle/input/law-qa/query_set_evaluate.json", "r") as file:
    for line in file:
        data = json.loads(line)
        no_segment_queries.append(data)

In [3]:
print(len(no_segment_queries))

18591


In [7]:
def isArticle(text):
    count = text.count('_')
    if count == 1:
        return True
    elif count == 2:
        return False
     
    return False

In [8]:
countArticle = 0
all_ref = 0
for query in no_segment_queries:
    for relevant_doc in query['relevant_docs']:
        if isArticle(relevant_doc):
            countArticle += 1
        all_ref += 1

In [10]:
print(countArticle, all_ref)

12292 20655


In [16]:
count_dict = {}
for query in no_segment_queries:
        num_docs = len(query.get('relevant_docs', []))
        if num_docs in count_dict:
            count_dict[num_docs] += 1
        else:
            count_dict[num_docs] = 1
            
print(count_dict)

{1: 16570, 2: 1979, 3: 41, 4: 1}


In [22]:
import re
def normalize_doc(doc):
    return re.sub(r'_\d+$', '', doc)

In [24]:
unique_releventdocs_set = set()
for query in no_segment_queries:
    normalized_docs = [normalize_doc(doc) for doc in query.get('relevant_docs', [])]
    unique_releventdocs_set.update(normalized_docs)

print(len(unique_releventdocs_set))

2684


In [21]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(no_segment_queries, test_size=0.2, random_state=42)

print(len(train_set))
print(len(test_set))

14872
3719
